In [1]:
import numpy as np
import pandas as pd
import string 
from string import digits
import re
import nltk
from nltk.corpus import stopwords
from sklearn import preprocessing
import itertools

In [2]:
import scipy
import csv
from nltk.tokenize import word_tokenize
import re, csv, yaml
from nltk.corpus import wordnet as wn
from nltk.metrics import edit_distance
nltk.download('punkt')
import itertools
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\praga\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\praga\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [126]:
#nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [3]:
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [3]:
imfile = pd.read_csv("AIT624-imgdataset.csv")

In [4]:
imfile.dropna(axis=0,how="all",inplace = True)

In [6]:
imfile['Concept'].head(5)

0     [business=1, ceremony=1, festival=1, group=1,...
1     [mountain=6, outdoors=6, travel=6, tree=6, fo...
2     [people=4, city=3, politics=3, architecture=2...
3     [people=28, politics=18, protest=13, travel=1...
4     [automobile=1, business=1, police=1, transpor...
Name: Concept, dtype: object

In [7]:
def convert_dict(s):
    return re.sub(r'\W',' ',s)

In [8]:
imfile['list'] = imfile['Concept'].apply(convert_dict)

In [9]:
imfile['list'].head(5)

0      business 1  ceremony 1  festival 1  group 1 ...
1      mountain 6  outdoors 6  travel 6  tree 6  fo...
2      people 4  city 3  politics 3  architecture 2...
3      people 28  politics 18  protest 13  travel 1...
4      automobile 1  business 1  police 1  transpor...
Name: list, dtype: object

In [10]:
def remove_num(s):
    s = re.sub("\d+"," ", s) 
    return " ".join(s.split())

In [11]:
imfile['list'] = imfile['list'].apply(remove_num)

In [12]:
##
imfile['list'].head(5)

0    business ceremony festival group indoors peopl...
1    mountain outdoors travel tree forest landscape...
2    people city politics architecture building con...
3    people politics protest travel adult nobody ou...
4    automobile business police transportation vehicle
Name: list, dtype: object

In [13]:
word_tokenize(imfile['list'][0])

['business',
 'ceremony',
 'festival',
 'group',
 'indoors',
 'people',
 'performance',
 'religion',
 'restaurant']

In [14]:
imfile['list'].tail(20)

372    adult cabinet facial expression furniture girl...
373    people politics travel coast group outdoors ac...
374    road people action street transportation trave...
375                                                     
376    city travel people architecture street politic...
377    people architecture travel building outdoors a...
378             adult clothing one people portrait women
379    nobody architecture garden house luxury modern...
380    beach coast leisure recreation relaxation reso...
381    architecture building business city cityscape ...
382    travel architecture business nobody building p...
383    adult architecture building city competition c...
384    adult bar beverage cafe commerce counter desk ...
385    carpet chair dining table easy chair fireplace...
386    people politics adult men police protest road ...
387    people recreation swimming water swimming pool...
388    adult people outdoors women portrait politics ...
389    dwelling furniture house

In [27]:
n=392
count=0
newlist = []
while(count<n):
    tokens = word_tokenize(imfile['list'][count])
    for a in range(0, len(tokens)-2):
        for b in range(a+1, len(tokens)-1):
            #print(a)
            #print(b)
            x = wn.synsets(tokens[a])
            y = wn.synsets(tokens[b])
            #print(x)
            #print(y)
            if (len(x) != 0) and (len(y) != 0):            
                ai = x[0]
                bi = y[0]
                xyz = ai.wup_similarity(bi)
                if xyz is None:
                    xyz = 0
                if xyz > 0.5:
                    tokens[b] = tokens[a]
                    #tokens = list(set(tokens))
                    #len(tokens)
                #print(ai)
                #print(bi)
                #print(xyz)
    #print(tokens)
    newlist.append(tokens)
    #print(newlist)
    #print(len(tokens))
    count+=1

# Lemmatisation

In [16]:
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [17]:
port = PorterStemmer()
wordnet_lemma = WordNetLemmatizer()

In [18]:
#wordnet_lemmatizer.lemmatize("dogs")

In [19]:
def normalize(s):
    stem = [port.stem(i) for i in s]
    lem = [wordnet_lemma.lemmatize(i,pos ='n') for i in stem]
    return ",".join([wordnet_lemma.lemmatize(i,pos ='v') for i in lem])
    #return lem   

In [20]:
imfile['list'] = imfile['list'].apply(normalize)
#imfile['list'] = imfile['list'].apply(split)

# Since Stemming and Lemmentization doesn't give proper result So we do LSA

# LSA using Sci-Kit Learn

In [22]:
#concept = imfile['Concept'].str.cat(sep=', ') #merging the columns
#concept = concept.replace("\\", "\\\\") #converting to raw document
#concept

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer


vectorizer = TfidfVectorizer(max_df=0.5, max_features=10000,min_df= 1, stop_words='english',use_idf=True, smooth_idf = True,analyzer='word')
X_train_tfidf = vectorizer.fit_transform(imfile['list'])
X_train_tfidf[0]


<1x1067 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements in Compressed Sparse Row format>

In [24]:
X_train_tfidf

<392x1067 sparse matrix of type '<class 'numpy.float64'>'
	with 12309 stored elements in Compressed Sparse Row format>

In [32]:
lsa = TruncatedSVD(n_components =393)
X_lsa = lsa.fit(X_train_tfidf)
X_lsa

TruncatedSVD(algorithm='randomized', n_components=393, n_iter=5,
       random_state=None, tol=0.0)

In [33]:
lsa.components_

array([[ 0.00133409,  0.01223909,  0.00122176, ...,  0.00826164,
         0.00539557,  0.00097392],
       [-0.00127506, -0.00126453,  0.00642366, ...,  0.02485697,
        -0.00129329,  0.0007609 ],
       [ 0.00289255,  0.01014152, -0.00132307, ..., -0.00765521,
         0.00468253,  0.00317075],
       ..., 
       [ 0.00486597, -0.02425703,  0.0032891 , ...,  0.04744442,
         0.00771466, -0.00170934],
       [-0.00121957, -0.04690714, -0.00879716, ...,  0.01761861,
        -0.00872048,  0.00337087],
       [ 0.01654213, -0.00573071, -0.03438837, ..., -0.00646766,
        -0.02040382,  0.00063886]])

In [45]:
def syn(word, lch_threshold=2.26):
    for net1 in wn.synsets(word):
        for net2 in wn.all_synsets():
            try:
                lch = net1.lch_similarity(net2)
            except:
                continue
            # The value to compare the LCH to was found empirically.
            # (The value is very application dependent. Experiment!)
            if lch >= 0.5:
                yield (net1, net2, lch)

In [16]:
def sim(word1, word2, lch_threshold=2.15, verbose=False):
    """Determine if two (already lemmatized) words are similar or not.

    Call with verbose=True to print the WordNet senses from each word
    that are considered similar.

    The documentation for the NLTK WordNet Interface is available here:
    http://nltk.googlecode.com/svn/trunk/doc/howto/wordnet.html
    """
    from nltk.corpus import wordnet as wn
    results = []
    for net1 in wn.synsets(word1):
        for net2 in wn.synsets(word2):
            try:
                lch = net1.lch_similarity(net2)
            except:
                continue
            # The value to compare the LCH to was found empirically.
            # (The value is very application dependent. Experiment!)
            if lch >= lch_threshold:
                results.append((net1, net2))
    if not results:
        return False
    if verbose:
        for net1, net2 in results:
            print(net1)
            print(net1.definition)
            print(net2)
            print(net2.definition)
            print('path similarity:')
            print(net1.path_similarity(net2))
            print('lch similarity:')
            print(net1.lch_similarity(net2))
            print('wup similarity:')
            print(net1.wup_similarity(net2))
            print('-' * 79)
    return True

In [17]:
for a, b in itertools.combinations(abc, 2):
    sim(a, b, verbose= False)
    

NameError: name 'abc' is not defined

In [18]:
def lange(abc):
    for a, b in itertools.combinations(abc, 2):
        ai = wn.synsets(a)[0]
        bi = wn.synsets(b)[0]
        print(ai)
        print(bi)
        xyz = ai.wup_similarity(bi)
        if xyz is None:
            xyz = 0
        xyz = [float(xyz)]
        print(xyz)

In [19]:
rc = pd.read_csv('RedCon.csv',delimiter= ',')

In [20]:
rc.head()

,Concept_wup,Concept_lch
0,"[business, ceremony, festival, business, indoo...","[business, ceremony, festival, business, indoo..."
1,"[mountain, outdoors, travel, tree, forest, lan...","[mountain, outdoors, travel, tree, forest, lan..."
2,"[people, city, politics, architecture, archite...","[people, city, politics, architecture, archite..."
3,"[people, politics, protest, protest, adult, ad...","[people, politics, protest, travel, adult, adu..."
4,"[automobile, business, business, automobile, v...","[automobile, business, business, transportatio..."


In [21]:
def lenth(line):
    abc = word_tokenize(line)
    abc =  list(set(abc))
    return len(abc)

In [22]:
rc['Concept_wup'] = rc['Concept_wup'].apply(convert_dict)
rc['Concept_lch'] = rc['Concept_lch'].apply(convert_dict)

In [23]:
rc['concept'] = imfile['list'].apply(lenth)
rc['wup_len'] = rc['Concept_wup'].apply(lenth)
rc['lch_len'] = rc['Concept_lch'].apply(lenth)
rc['Concepts'] = imfile['Concept']

In [233]:
pd.set_option('display.max_colwidth', -1)

In [29]:
rc.head(5)

,Concept_wup,Concept_lch,concept,wup_len,lch_len,Concepts
0,business ceremony festival business indoo...,business ceremony festival business indoo...,9,6,7,"[business=1, ceremony=1, festival=1, group=1,..."
1,mountain outdoors travel tree forest lan...,mountain outdoors travel tree forest lan...,22,14,18,"[mountain=6, outdoors=6, travel=6, tree=6, fo..."
2,people city politics architecture archite...,people city politics architecture archite...,37,15,27,"[people=4, city=3, politics=3, architecture=2..."
3,people politics protest protest adult ad...,people politics protest travel adult adu...,110,26,51,"[people=28, politics=18, protest=13, travel=1..."
4,automobile business business automobile v...,automobile business business transportatio...,5,3,4,"[automobile=1, business=1, police=1, transpor..."
